In [1]:
import torch
from evaluate import load

from pathlib import Path
from transformers import SegformerImageProcessor
from transformers import SegformerForSemanticSegmentation

from tqdm import tqdm

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import sys
sys.path.append('..')

from training.dataset import SemanticSegmentationDataset

/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
idx2label = ['Barren', 'Forest', 'Agriculture', 'Road', 'Building', 'Water']

processor = SegformerImageProcessor(reduce_labels=True)
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b0", num_labels=len(idx2label))

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
img_dir = '/home/a-ploskin/repos/TerraLabel/data/task_0/data'
masks_dir = '/home/a-ploskin/repos/TerraLabel/data/masks'

train_ds, eval_ds = SemanticSegmentationDataset.get_train_and_eval_datasets(
    processor, img_dir, masks_dir
)

In [4]:
BATCH_SIZE = 8

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
LR = 0.00006

bias_params = [p for name, p in model.named_parameters() if 'bias' in name]
others = [p for name, p in model.named_parameters() if 'bias' not in name]

optimizer = torch.optim.AdamW(
    [{'params': others},
    {'params': bias_params, 'weight_decay': 0}],
    lr=LR
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cuda:1'
print(f"Device: {device}")
model.to(device)

log_path = './logs'
writer = SummaryWriter(log_path)
metric = load("mean_iou")

Device: cuda:1


In [6]:
@torch.no_grad()
def eval_epoch(model, val_loader, writer, metric, device):
    epoch_loss = []

    model.eval()

    for batch in val_loader:
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        epoch_loss.append(outputs.loss.cpu().item())

        upsampled_logits = torch.nn.functional.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
        predicted = upsampled_logits.argmax(dim=1)
        metric.add_batch(predictions=predicted.detach().cpu().numpy(), references=labels.detach().cpu().numpy())

    metrics = metric.compute(
        num_labels=6,
        ignore_index=255,
        reduce_labels=False, # we've already reduced the labels before)
    )

    return sum(epoch_loss) / len(val_loader), metrics["mean_iou"], metrics["mean_accuracy"]

def train_epoch(model, optimizer, train_loader, writer, epoch, device):
    train_loss_iter = []
    epoch_loss = []

    model.train()

    for idx, batch in enumerate(train_loader):
        pixel_values = batch["pixel_values"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss, logits = outputs.loss, outputs.logits

        loss.backward()
        optimizer.step()

        train_loss_iter.append(loss.detach().cpu().item())
        epoch_loss.append(loss.detach().cpu().item())

        writer.add_scalar("Train/loss_step", train_loss_iter[-1], idx + epoch * len(train_loader))
        writer.add_scalar("Train/epoch", epoch + 1, idx + epoch * len(train_loader))

    return train_loss_iter, sum(epoch_loss) / len(epoch_loss)

def train(model, optimizer, train_loader, val_loader, writer, metric, device):
    train_loss_iter = []
    train_loss_epoch = []

    eval_iou = []
    eval_acc = []
    eval_loss = []

    for epoch in tqdm(range(30)):
        print("Epoch: ", epoch)

        cur_train_loss_iter, train_epoch_loss = train_epoch(model, optimizer, train_loader, writer, epoch, device)

        train_loss_iter += cur_train_loss_iter
        train_loss_epoch.append(train_epoch_loss)

        writer.add_scalar("Train/loss_epoch", train_loss_epoch[-1], epoch + 1)

        val_epoch_loss, iou, acc = eval_epoch(model, val_loader, writer, metric, device)

        eval_iou.append(iou)
        eval_acc.append(acc)
        eval_loss.append(val_epoch_loss)

        writer.add_scalar("Eval/IoU", iou, epoch + 1)
        writer.add_scalar("Eval/Accuracy", acc, epoch + 1)
        writer.add_scalar("Eval/loss", eval_loss[-1], epoch + 1)

        print("Mean_iou:", iou)
        print("Mean accuracy:", acc)
        print("Loss:", train_loss_epoch[-1])


    return train_loss_iter, train_loss_epoch, eval_iou, eval_acc, eval_loss

In [7]:
train_loss_iter, train_loss_epoch, eval_iou, eval_acc, eval_loss = train(model, optimizer, train_loader, test_loader, writer, metric, device)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch:  0


/home/a-ploskin/repos/TerraLabel/venv/lib/python3.12/site-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")
  3%|▎         | 1/30 [00:28<13:48, 28.56s/it]

Mean_iou: 0.19283736428224366
Mean accuracy: 0.35439416703225385
Loss: 1.756628155708313
Epoch:  1


  3%|▎         | 1/30 [00:42<20:38, 42.72s/it]


KeyboardInterrupt: 